In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import statsmodels as sm

import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots

import matplotlib.path as mpath
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection

import dash
import dash_core_components as dcc
import dash_html_components as html

import json
import os

import folium
from folium import IFrame

pd.set_option('float_format', '{:,.2f}'.format)
np.set_printoptions(precision=3)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

from math import sqrt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm

from sklearn.cluster import KMeans 

# Part 1: Preprocessing

### FSI

In [3]:
df_2019 = pd.read_excel('FSI_DATA/fsi-2019.xlsx')
df_2018 = pd.read_excel('FSI_DATA/fsi-2018.xlsx')
df_2017 = pd.read_excel('FSI_DATA/fsi-2017.xlsx')
df_2016 = pd.read_excel('FSI_DATA/fsi-2016.xlsx')
df_2015 = pd.read_excel('FSI_DATA/fsi-2015.xlsx')
df_2014 = pd.read_excel('FSI_DATA/fsi-2014.xlsx')
df_2013 = pd.read_excel('FSI_DATA/fsi-2013.xlsx')
df_2012 = pd.read_excel('FSI_DATA/fsi-2012.xlsx')
df_2011 = pd.read_excel('FSI_DATA/fsi-2011.xlsx')
df_2010 = pd.read_excel('FSI_DATA/fsi-2010.xlsx')
df_2009 = pd.read_excel('FSI_DATA/fsi-2009.xlsx')
df_2008 = pd.read_excel('FSI_DATA/fsi-2008.xlsx')
df_2007 = pd.read_excel('FSI_DATA/fsi-2007.xlsx')
df_2006 = pd.read_excel('FSI_DATA/fsi-2006.xlsx')

In [4]:
frames = [df_2006, df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013,
          df_2014, df_2015, df_2016, df_2017, df_2018, df_2019]

df_fsi = pd.concat(frames, sort=False)

df_fsi['Year'] = df_fsi['Year'].dt.to_period('Y')
df_fsi['Year'] = df_fsi['Year'].astype(str)
df_fsi['Year'] = pd.to_datetime(df_fsi['Year'], format='%Y')

df_fsi.rename(columns={'Total':'FSI'}, inplace=True)

### V-Dem

In [5]:
df_vd = pd.read_csv('Data/V-Dem/V-Dem-CY-Full+Others.csv')

C:\Users\Diego\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning:

Columns (507,509,510,793,794,1055,2791,3007,3008,3063,3065,3069,3070,3072,3073,3833,3834,3836) have mixed types. Specify dtype option on import or set low_memory=False.



In [6]:
df_vd = df_vd[['country_name', 'country_text_id', 'year',
               'v2x_polyarchy', 'v2x_libdem', 'v2x_partipdem', 'v2x_delibdem', 'v2x_egaldem']]

col = df_vd.loc[:, 'v2x_polyarchy':'v2x_egaldem']
df_vd['VDem'] = col.mean(axis=1)

df_vd['year'] = pd.to_datetime(df_vd['year'], format='%Y')

df_vd = df_vd.rename(columns={'country_name':'Country', 'year':'Year'})

### HDI

In [7]:
df_hdi = pd.read_excel('Data/Human Development Index (HDI).xlsx')

In [8]:
df_hdi = df_hdi.drop(['HDI Rank (2017)'], axis=1)
df_hdi = df_hdi.set_index('Country')
df_hdi = df_hdi.unstack().reset_index(level=0, drop=False)
df_hdi = df_hdi.rename(columns={'level_0':'Year', 0:'HDI'}).reset_index()

df_hdi['Country'] = df_hdi['Country'].replace({'Venezuela (Bolivarian Republic of)':'Venezuela',
                                               'Bolivia (Plurinational State of)':'Bolivia'})

df_hdi['Year'] = pd.to_datetime(df_hdi['Year'], format='%Y')

### Merge

In [9]:
df_1 = df_fsi.set_index(['Country', 'Year'])
df_2 = df_vd.set_index(['Country', 'Year'])
df_3 = df_hdi.set_index(['Country', 'Year'])

df = df_1.merge(df_2, left_index=True, right_index=True)
df = df.merge(df_3, left_index=True, right_index=True)
df = df.reset_index()

df['Year'] = pd.DatetimeIndex(df['Year']).year

In [10]:
df = df[['Country', 'Year', 'FSI', 'C1: Security Apparatus',
       'C2: Factionalized Elites', 'C3: Group Grievance', 'E1: Economy',
       'E2: Economic Inequality', 'E3: Human Flight and Brain Drain',
       'P1: State Legitimacy', 'P2: Public Services', 'P3: Human Rights',
       'S1: Demographic Pressures', 'S2: Refugees and IDPs',
       'X1: External Intervention', 'v2x_polyarchy', 'v2x_libdem',
       'v2x_partipdem', 'v2x_delibdem', 'v2x_egaldem', 'VDem', 'HDI']]

df['HDI'] = pd.to_numeric(df['HDI'],errors='coerce')

df = df.sort_values(by=['Country', 'Year'])

### G.1: HDI (selected countries)

In [11]:
countries = ['Argentina', 'Bolivia', 'Brazil', 'Canada', 'Chile', 
             'Colombia', 'Ecuador', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela']

In [12]:
dff = df.loc[df['Country'].isin(countries)]
dff = dff[['Country', 'Year', 'FSI', 'VDem', 'HDI']]
dff.head()

,Country,Year,FSI,VDem,HDI
1214,Argentina,2006,40.80,0.66,0.79
592,Argentina,2007,41.40,0.64,0.79
975,Argentina,2008,41.40,0.61,0.80
1517,Argentina,2009,44.70,0.62,0.80
1204,Argentina,2010,45.80,0.60,0.81


In [13]:
def f_HDI(countries):
    
    '''
    With this function it is possible compare HDI between multiple countries
    '''
    
    data = []
    
    for country in countries:
        
        dff_t = dff.loc[dff['Country']==country]
        
        trace1 = go.Scatter(x=dff_t['Year'],
                            y=dff_t['HDI'],
                          mode='lines',
                          name=country)
        
        data.append(trace1)
        
    layout = go.Layout(title='HDI (selected countries)')

    fig = go.Figure(data=data, layout=layout)
        
    return pyo.plot(fig)

In [14]:
f_HDI(countries)

'temp-plot.html'

In [15]:
def f_HDI_var(countries):
    
    '''
    With this function it is possible compare HDI between multiple countries
    '''
    
    data = []
    
    for country in countries:
        
        dff_t = dff.loc[dff['Country']==country]
        V1 = dff_t.iloc[0,4]
        V2 = dff_t.iloc[-1,4]
        R = np.around(((V2-V1)/V1)*100, decimals=2)
        data.append([country, R])
        Table = pd.DataFrame(data, columns=['Country', 'HDI Var(%)'])
   
    return Table.sort_values(by='HDI Var(%)', ascending=False).set_index('Country')

In [16]:
f_HDI_var(countries)

,HDI Var(%)
Country,
Bolivia,10.00
Paraguay,8.17
Brazil,8.12
Colombia,8.10
Ecuador,8.05
Peru,7.91
Chile,6.04
Uruguay,5.51
Argentina,4.83


### G2: Correlations

In [17]:
df[['FSI', 'VDem', 'HDI']].corr()

,FSI,VDem,HDI
FSI,1.00,-0.73,-0.85
VDem,-0.73,1.00,0.55
HDI,-0.85,0.55,1.00


In [18]:
w = df.query("Year==2017")

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=w['FSI'], 
               y=w['HDI'],
               mode='markers',
               name='FSI'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=w['VDem'], 
               y=w['HDI'],
               mode='markers',
               name='VDem'),
    row=1, col=2
)

fig.update_layout(title_text="Relación entre HDI/FSI y HDI/VDem")
pyo.plot(fig)

'temp-plot.html'

### G3: Country Analysis

In [89]:
country = 'Argentina'

df_Normalize = df.loc[df['Country']==country]
df_Normalize = df_Normalize[['Country', 'Year', 'FSI', 'HDI', 'VDem']]

# Normalize the Data
scaler = MinMaxScaler(feature_range=(0, 1))
columns = ['FSI', 'HDI', 'VDem']
df_Normalize[columns] = scaler.fit_transform(df_Normalize.loc[:, columns])

df_Normalize[df_Normalize['Country']==country]

,Country,Year,FSI,HDI,VDem
1214,Argentina,2006,0.00,0.00,1.00
592,Argentina,2007,0.08,0.13,0.63
975,Argentina,2008,0.08,0.21,0.18
1517,Argentina,2009,0.51,0.32,0.34
1204,Argentina,2010,0.66,0.68,0.02
1650,Argentina,2011,0.79,0.84,0.24
1179,Argentina,2012,0.75,0.82,0.49
426,Argentina,2013,0.70,0.87,0.00
582,Argentina,2014,0.86,0.87,0.21
155,Argentina,2015,0.89,0.92,0.32


In [90]:
def f_country_analysis(country):
    
    df_t = df.loc[df['Country']==country]
    df_t = df_t[['Country', 'Year', 'FSI', 'HDI', 'VDem']]

    # Normalize the Data
    scaler = MinMaxScaler(feature_range=(0, 1))
    columns = ['FSI', 'HDI', 'VDem']
    df_t[columns] = scaler.fit_transform(df_t.loc[:, columns])

    trace1 = go.Scatter(x=df_t['Year'],
                        y=df_t['FSI'],
                      mode='lines',
                      name='FSI')

    trace2 = go.Scatter(x=df_t['Year'],
                        y=df_t['VDem'],
                      mode='lines',
                      name='VDem')

    trace3 = go.Scatter(x=df_t['Year'],
                        y=df_t['HDI'],
                      mode='lines',
                      name='HDI')

    data = [trace1, trace2, trace3]

    layout = go.Layout(title='Fragilidad del Estado, Calidad de la Democracia y Desarrollo Humano: {}'.format(country))

    fig = go.Figure(data=data, layout=layout)

    return pyo.plot(fig)

In [91]:
f_country_analysis('Argentina')

'temp-plot.html'

### FSI analysis

In [42]:
def f_country_fsi(country, initial_year, final_year):
    dff = df[df['Country']==country]
    dff = dff.loc[:,'Country':'X1: External Intervention']
    dff = dff.drop(['Country', 'FSI'], axis=1)
    dff['Year'] = pd.to_datetime(dff['Year'], format='%Y').dt.to_period('Y')
    dff = dff.set_index('Year')
    dff = dff[(dff.index >=initial_year) & (dff.index <= final_year)]
    dff = dff.T
    dff['VAR'] = ((dff[final_year] - dff[initial_year])/dff[initial_year])*100
    #dff = dff.sort_values(by='VAR', ascending=False)
    
        
    return dff

In [96]:
radar_1 = f_country_fsi('Argentina', '2006', '2008')

In [97]:
radar_2 = f_country_fsi('Argentina', '2008', '2017')

In [98]:
# Normalize the Data
scaler = MinMaxScaler(feature_range=(-1, 1))
columns = ['VAR']
radar_1[columns] = scaler.fit_transform(radar_1.loc[:,columns])
radar_2[columns] = scaler.fit_transform(radar_2.loc[:,columns])

In [99]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=radar_1['VAR'],
      theta=radar_1.index,
      fill='toself',
      name='2007 - 2012'
))

fig.add_trace(go.Scatterpolar(
      r=radar_2['VAR'],
      theta=radar_2.index,
      fill='toself',
      name='2012 - 2017'
))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[-1, 1]
    )),
  showlegend=True, legend=dict(x=0.75, y=-0.1)
)


pyo.plot(fig)

'temp-plot.html'

### Democracy analysis

In [66]:
def f_country_vdem(country, initial_year, final_year):
    dff = df[df['Country']==country]
    dff = dff.loc[:,['Country', 'Year', 'v2x_polyarchy', 'v2x_libdem', 'v2x_partipdem', 'v2x_delibdem', 'v2x_egaldem']]
    dff = dff.rename(columns={'v2x_polyarchy':'Poliarchy', 'v2x_libdem':'Liberal democracy',
                              'v2x_partipdem':'Participatory democracy', 'v2x_delibdem':'Deliberative democracy',
                              'v2x_egaldem':'Egalitarian democracy'})
    dff = dff.drop(['Country'], axis=1)
    dff['Year'] = pd.to_datetime(dff['Year'], format='%Y').dt.to_period('Y')
    dff = dff.set_index('Year')
    dff = dff[(dff.index >=initial_year) & (dff.index <= final_year)]
    dff = dff.T
    dff['VAR'] = ((dff[final_year] - dff[initial_year])/dff[initial_year])*100
    #dff = dff.sort_values(by='VAR', ascending=False)
    
        
    return dff

In [100]:
radar_3 = f_country_vdem('Argentina', '2007', '2012')

In [101]:
radar_4 = f_country_vdem('Argentina', '2012', '2017')

In [102]:
# Normalize the Data
scaler = MinMaxScaler(feature_range=(-1, 1))
columns = ['VAR']
radar_3[columns] = scaler.fit_transform(radar_3.loc[:,columns])
radar_4[columns] = scaler.fit_transform(radar_4.loc[:,columns])

In [103]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=radar_3['VAR'],
      theta=radar_3.index,
      fill='toself',
      name='2007 - 2012'
))

fig.add_trace(go.Scatterpolar(
      r=radar_4['VAR'],
      theta=radar_4.index,
      fill='toself',
      name='2012 - 2017'
))


fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[-20, 5]
    )),
  showlegend=True, legend=dict(x=0.75, y=-0.1)
)


pyo.plot(fig)

'temp-plot.html'

# Part 2: Classification

In [203]:
df.tail()

,Country,Year,FSI,C1: Security Apparatus,C2: Factionalized Elites,C3: Group Grievance,E1: Economy,E2: Economic Inequality,E3: Human Flight and Brain Drain,P1: State Legitimacy,...,S1: Demographic Pressures,S2: Refugees and IDPs,X1: External Intervention,v2x_polyarchy,v2x_libdem,v2x_partipdem,v2x_delibdem,v2x_egaldem,VDem,HDI
807,Zimbabwe,2013,105.20,8.40,9.70,8.40,8.60,8.60,8.60,9.20,...,9.20,8.70,7.80,0.30,0.20,0.19,0.17,0.15,0.20,0.52
1435,Zimbabwe,2014,102.80,8.10,10.00,8.10,8.30,8.30,8.30,9.30,...,8.90,8.40,7.70,0.30,0.20,0.16,0.19,0.16,0.20,0.53
1407,Zimbabwe,2015,100.00,7.90,9.70,7.80,8.00,8.10,8.00,9.00,...,8.70,8.40,7.60,0.31,0.20,0.16,0.19,0.16,0.20,0.53
673,Zimbabwe,2016,100.50,7.80,9.80,7.50,8.30,8.20,8.10,8.90,...,8.60,8.70,7.70,0.30,0.19,0.18,0.15,0.13,0.19,0.53
1181,Zimbabwe,2017,101.60,8.10,9.80,7.30,8.60,8.50,7.90,9.20,...,9.10,8.50,7.50,0.31,0.22,0.20,0.19,0.16,0.22,0.54


In [204]:
df_norm = df[['Country', 'Year', 'FSI', 'VDem', 'HDI']]


# Clean the Data
df_norm = df_norm.loc[df['Year']==2007]
df_norm = df_norm.drop(['Year'], axis=1)
df_norm = df_norm.set_index('Country')

# Normalize the Data
X = df_norm.values[:,1:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)

In [205]:
clusterNum = 5
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12)
k_means.fit(X)
labels = k_means.labels_

In [206]:
df_norm["Clus_km"] = labels

In [207]:
df_norm.head()

,FSI,VDem,HDI,Clus_km
Country,,,,
Afghanistan,102.30,0.26,0.43,1
Albania,70.50,0.45,0.72,0
Algeria,75.90,0.23,0.71,2
Angola,84.90,0.11,0.49,1
Argentina,41.40,0.64,0.79,3


In [208]:
def f_scatter_variables(V1, V2):    
    
    dff = df_norm
    dff.reset_index(inplace=True)

    fig = px.scatter(dff, x=V1, y=V2,
                     color='Clus_km', hover_name='Country')

  
    fig.update_layout(title=go.layout.Title(text='FSI'), showlegend=False)

    pyo.plot(fig)   

In [209]:
df_norm.columns

Index(['FSI', 'VDem', 'HDI', 'Clus_km'], dtype='object')

In [211]:
f_scatter_variables('HDI', 'FSI')

In [197]:
f_scatter_variables('HDI', 'VDem')

# Cluster groups

In [198]:
cluster_0 = df_norm.loc[df_norm['Clus_km']==0]
cluster_0['Country'] .unique()

array(['Algeria', 'Azerbaijan', 'Belarus', 'China', 'Cuba', 'Egypt',
       'Equatorial Guinea', 'Gabon', 'Kazakhstan', 'Libya', 'Malaysia',
       'Morocco', 'Saudi Arabia', 'Tajikistan', 'Thailand', 'Tunisia',
       'Uzbekistan', 'Venezuela'], dtype=object)

In [199]:
cluster_1 = df_norm.loc[df_norm['Clus_km']==1]
cluster_1['Country'] .unique() 

array(['Afghanistan', 'Angola', 'Bangladesh', 'Bhutan', 'Burundi',
       'Cameroon', 'Central African Republic', 'Chad', 'Eritrea',
       'Ethiopia', 'Guinea', 'Haiti', 'Mauritania', 'Nepal', 'Nigeria',
       'Pakistan', 'Rwanda', 'Sudan', 'Togo', 'Turkmenistan', 'Uganda',
       'Yemen', 'Zimbabwe'], dtype=object)

In [200]:
cluster_2 = df_norm.loc[df_norm['Clus_km']==2]
cluster_2['Country'] .unique() 

array(['Argentina', 'Australia', 'Belgium', 'Brazil', 'Costa Rica',
       'Denmark', 'France', 'Hungary', 'Latvia', 'Lithuania', 'Poland',
       'United Kingdom'], dtype=object)

In [201]:
cluster_3 = df_norm.loc[df_norm['Clus_km']==3]
cluster_3['Country'] .unique() 

array(['Albania', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Bulgaria', 'Colombia', 'Dominican Republic', 'Ecuador',
       'El Salvador', 'Guatemala', 'Honduras', 'India', 'Indonesia',
       'Iraq', 'Lebanon', 'Mexico', 'Paraguay', 'Peru', 'Philippines',
       'Turkey', 'Ukraine'], dtype=object)

In [202]:
cluster_4 = df_norm.loc[df_norm['Clus_km']==4]
cluster_4['Country'] .unique()

array(['Benin', 'Burkina Faso', 'Kenya', 'Liberia', 'Malawi',
       'Mozambique', 'Sierra Leone', 'Zambia'], dtype=object)

In [177]:
cluster_2.describe()

,index,FSI,VDem,HDI,Clus_km
count,32.00,32.00,32.00,32.00,32.00
mean,81.88,71.33,0.19,0.76,2.00
std,45.98,14.73,0.09,0.07,0.00
min,2.00,32.50,0.04,0.65,2.00
25%,46.75,65.28,0.11,0.71,2.00
50%,85.50,74.20,0.18,0.76,2.00
75%,120.75,77.73,0.27,0.80,2.00
max,143.00,105.40,0.35,0.93,2.00


In [178]:
cluster_1.describe()

,index,FSI,VDem,HDI,Clus_km
count,33.00,33.00,33.00,33.00,33.00
mean,68.36,33.50,0.76,0.89,1.00
std,40.13,11.32,0.05,0.05,0.00
min,4.00,18.70,0.63,0.79,1.00
25%,36.00,22.60,0.73,0.87,1.00
50%,65.00,32.40,0.77,0.90,1.00
75%,95.00,41.70,0.79,0.93,1.00
max,141.00,62.60,0.84,0.95,1.00


In [179]:
cluster_0.describe()

,index,FSI,VDem,HDI,Clus_km
count,19.00,19.00,19.00,19.00,19.00
mean,74.63,86.02,0.46,0.51,0.00
std,34.39,8.73,0.08,0.08,0.00
min,14.00,69.70,0.34,0.35,0.00
25%,56.50,79.80,0.40,0.44,0.00
50%,80.00,88.00,0.45,0.52,0.00
75%,97.50,91.95,0.52,0.58,0.00
max,120.00,101.60,0.59,0.64,0.00
